### Sequential vs Functional API

In [9]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, BatchNormalization

In [4]:
#Sequential Implemention 1
model =  keras.Sequential([
    layers.Dense(16, activation="relu", name="layer1"),
    layers.Dense(8, activation="relu", name="layer2"), 
    layers.Dense(4, name="layer3")])

In [6]:
#Sequential Implemention 2
model = keras.Sequential()
model.add(layers.Dense(16, activation="relu", name="layer1"))
model.add(layers.Dense(8, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

In [7]:
#Functional Implemention 2

#Encode an image to a vector
image_model = keras.Sequential([
    layers.Conv2D(128, (3,3), activation='relu', input_shape=(299,299,3)),
    layers.MaxPooling2D(),
    layers.Flatten()
])

img_input = layers.Input(shape=(299,299,3))
encoded_image = image_model(img_input)

In [8]:
#Language model
txt_input = layers.Input(shape=(200,), dtype='int32', name='context')
embed_output = layers.Embedding(input_dim=5000, output_dim=512, input_length=200)(txt_input)
encoded_txt = layers.LSTM(128)(embed_output)

In [ ]:
#Concatenation
concat = layers.concatenate([encoded_image, encoded_txt])
output_class = layers.Dense(512, activation='softmax')(concat)
final_model = keras.Model(inputs=[img_input, txt_input], outputs = output_class)

Here, the inception_block uses two branches of the CNN_block. The outputs coming from these two branches are then merged together by passing two instances of CNN_block as a list to the layers.concatenate layer.

 

This feature is called Shared layers, where you have reused the same block again inside our new block/lego. By sharing the information across two or more different inputs, shared layers allow you to train a model even on lesser data.

 

Now that you have defined all the blocks that would be used for the model, you can go ahead and build the architecture for the same. Consider the model that you want to build is the following.

(input shape: 28,28,1))

        ↓
[CNN_block(32 units)]

        ↓
[MaxPooling2D(3)]

        ↓
[Inception_block (32 units, 32 units)]

        ↓
[Inception_block (64 units, 64 units)]

        ↓
[CNN_block(128 units)]

        ↓
[GlobalAveragePooling2D()]

        ↓
[Dense (256 units, relu activation)]

        ↓
[Dropout(0.5)]

        ↓
[Dense (10 units)]

        ↓
(output: logits of a probability distribution over 10 classes


In [19]:
def CNN_block(x,filters, kernel_size=3, padding="same"):
    # Defining a function which returns an operation of Convolution, BatchNormalisation and Relu.
    # Here x is the input.
    x = Conv2D(filters, (3,3), padding=padding)(x)
    x = BatchNormalization()(x)
    return tf.nn.relu(x)

In [20]:
def inception_block(x, filter1, filter2):
    # Defining a function which applies two CNN blocks. The output of both these blocks are then concatenated
    conv1 = CNN_block(x, filter1)
    conv2 = CNN_block(x, filter2)
    x = keras.layers.concatenate([conv1, conv2])
    return x

In [21]:
inputs = keras.Input(shape=(28, 28, 1))
x = CNN_block(inputs,32)
x = layers.MaxPooling2D(3)(x)
x = inception_block(x,32, 32)
x = inception_block(x,64, 64)
x = CNN_block(x,128)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)
model = keras.Model(inputs, outputs, name="final_model")
model.summary()

Model: "final_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 28, 28, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 tf.nn.relu (TFOpLambda)        (None, 28, 28, 32)   0           ['batch_normalization[0

In [33]:
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [34]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, to_file='model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### Model Subclassing

In the previous segment, you have learnt how to implement Sequential and  Functional API.

Among these two, you might have observed that the Sequential API is the easiest approach to build models, but it’s also a naive approach there are many limitations when it comes to:

Building models with multiple inputs and outputs
Sharing layers
 

The Functional API bridge this gap by allowing you to create more complex models. This approach provides the support of multiple inputs and multiple outputs, layer sharing, branching and reusability. But in this approach for each block, we are applying operations on the built-in layers. The Keras API provides you with many built-in layers like:

Convolutional layers: Conv1D, Conv2D, etc.
Pooling layers: MaxPooling1D, MaxPooling2D, etc.
RNN layers: GRU, LSTM, etc.
BatchNormalization, Dropout, Embedding, etc.
 

But if you don't want to work with these built-in layers and you want to create your own layers, then the third approach - Model Subclassing provides an elegant solution.

 

Model Subclassing is tailor-made for advanced developers who want to customise how their model, layer, and training process works. Let's understand this in detail in the next video.